# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) are ignored in the calculations.

Replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):
  1. key=YOUR-LASTFM-API-KEY
  2. username=YOUR-LASTFM-USERNAME

In [6]:
import requests, json, time, pandas as pd
key="8b1214dc96c0119ad6d86829a57ceaa2"
username="adben"

In [7]:
# how long to pause between consecutive API requests
pause_duration = 0.5

## First get your all-time most played tracks, artists, and albums

In [8]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 0 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [9]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('data/lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,Ludwig van Beethoven,"Symphony No.7 in A, Op.92: 2. Allegretto",178
1,Radiohead,Weird Fishes/ Arpeggi,136
2,The Killers,Read My Mind,106
3,London Philharmonic Orchestra,"Orchestral Suite No. 3 in D major, BWV 1068: Air",97
4,Muzio Clementi,"Keyboard Sonatina In C Major, Op. 36, No. 1: I...",95


In [10]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('data/lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,The White Stripes,1196
1,The Dead Weather,811
2,Portishead,800
3,The Black Keys,695
4,Kasabian,678


In [12]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('data/lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,Kasabian,Kasabian,536
1,The Dead Weather,Sea of Cowards,461
2,The White Stripes,White Blood Cells,397
3,London Philharmonic Orchestra,The 50 Greatest Pieces of Classical Music,380
4,Norah Jones,Come Away with Me,302


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [13]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = int(response[method]['@attr']['totalPages'])
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print('{} total pages to retrieve'.format(total_pages))
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print(page, end=' ')
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [14]:
# get all scrobbled tracks ever, in order of recency (pages=0 to get all)
scrobbles = get_scrobbles(pages=0)

215 total pages to retrieve
10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 

In [15]:
# save the dataset
scrobbles.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print('{:,} total rows'.format(len(scrobbles)))
scrobbles.head()

42,807 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,Anne Akiko Meyers,91cffe48-b682-4e2a-ad90-b09633f4a8f7,Smile,,Arvo Part: Spiegel Im Spiegel (mirror In Mirror),,1485962499,2017-02-01 15:21:39
1,Camille Saint-Saëns,4137c070-15b2-4d00-a9f0-3517d02a9ba8,100 Classical Favourites,,Le carnaval des animaux: Le cygne,,1485962308,2017-02-01 15:18:28
2,Ludwig van Beethoven,1f9df192-a621-4f54-8850-2c5373b7eac9,The King's Speech OST,,Speaking Unto Nations (Beethoven Symphony no 7...,,1485962007,2017-02-01 15:13:27
3,Remo Giazotto,d6f85c05-93a6-4c1a-bcd9-84d712bf0cff,Albinoni / Barber / Handel (Uk),,Adagio in G Minor (attrib. to Albinoni),3350db72-5a90-4324-9a61-15b7f02391f7,1485961314,2017-02-01 15:01:54
4,Tomaso Giovanni Albinoni,83903121-f611-4875-984c-673ae7173e56,Great Baroque Favorites: Pachelbel's Canon,,Adagio in G minor,2809df55-d82f-4528-b2ec-7f9fa3f23794,1485960812,2017-02-01 14:53:32
